# FaaSET Notebook

This Jupyter Notebook provides an interactive platform for FaaS function development, testing, running experiments, and processing results.

In [ ]:
import FaaSET
import FaaSRunner

# Functions

Any function with the @cloud_function decorator will be uploaded to the cloud. Define platforms and memory settings in the decorator. 
Functions are tested locally and must run sucessfully before being deployed.

In [ ]:
@FaaSET.cloud_function(platform="AWS")
def hello_world_faaset(request, context): 
    from SAAF import Inspector
    inspector = Inspector()
    inspector.inspectCPUInfo()
    inspector.addAttribute("message", "Hello from the hotel lambda " + str(request["name"]) + "!")
    return inspector.finish()

def hello_world_local(request, context): 
    from SAAF import Inspector
    inspector = Inspector()
    inspector.inspectCPUInfo()
    inspector.addAttribute("message", "Hello from your computer " + str(request["name"]) + "!") 
    return inspector.finish()

# Run our local hello_world function and check the CPU.
local = hello_world_local({"name": "Steve"}, None)

# Run our cloud hello_world function and check the CPU.
cloud = hello_world_faaset({"name": "Steve"}, None)

print("\n----- Results -----\n")
print("Local CPU: " + local['cpuType'])
print("Cloud CPU: " + cloud['cpuType'])

In [ ]:
@FaaSET.cloud_function(platform="AWS", config={"storage": 1024})
def disk_writer_faaset(request, context):
    from SAAF import Inspector
    import random
    import string
    import os 
    
    inspector = Inspector()
    inspector.inspectAll() 
    
    # Generate really long string in memory (Kinda fast?). 
    data = "1234567890"
    for x in range(23):
        data += data
    
    # Repeatedly write it to a file and delete it.
    for i in range(request["loops"]):
        
        # Write file...
        write_file = open("/tmp/text" + str(i) + ".txt", "w")
        a = write_file.write(data)
        write_file.close()
        
        # Delete the file.
        os.remove("/tmp/text" + str(i) + ".txt")
    
    inspector.inspectAllDeltas()
    return inspector.finish()

disk_writer_faaset({"loops": 3}, None)

In [ ]:
@FaaSET.cloud_function(platform="AWS", config={"memory": 1024})
def calc_service_faaset(request, context):
    from SAAF import Inspector
    from random import randint
    import time 
    from threading import Thread

    inspector = Inspector()
    inspector.inspectAll() 

    def do_calcs(calcs, loops):
        operand_a = [0] * calcs
        operand_b = [0] * calcs
        operand_c = [0] * calcs

        for k in range(0, loops):
            for i in range(0, calcs):
                j = randint(0, calcs - 1)
                operand_a[j] = randint(0, 99999)
                operand_b[j] = randint(0, 99999)
                operand_c[j] = randint(0, 99999)
                mult = operand_a[j] * operand_b[j]

    threads = request["threads"]
    calcs = request["calcs"]
    loops = request["loops"]
    threadList = []
    for i in range(0, threads):
        thread = Thread(target=do_calcs, args=(calcs, loops))
        thread.start()
        threadList.append(thread)
    for i in range(len(threadList)):
        threadList[i].join()

    inspector.inspectAllDeltas()
    return inspector.finish()

calc_service_faaset({"threads": 2, "calcs": 10000, "loops": 10})

# Example Functions with Dependencies

In [ ]:
# Install igraph dependency to local environment and
# to function source code directory.
%pip install python-igraph
!mkdir ./functions/page_rank_faaset
!python3.8 -m pip install igraph -t ./functions/page_rank_faaset

@FaaSET.cloud_function(platform="AWS")
def page_rank_faaset(request, context):
    from SAAF import Inspector 
    import datetime 
    import igraph
    import time
    
    inspector = Inspector()
    inspector.inspectAll()  
    
    size = request.get('size')  
    loops = request.get('loops')

    for x in range(loops):
        graph = igraph.Graph.Tree(size, 10)
        result = graph.pagerank()  

    inspector.inspectAllDeltas()
    return inspector.finish()

page_rank_faaset({"size": 10000, "loops": 5}, None)

# Execute Experiments

Use FaaS Runner to execute complex FaaS Experiments.

In [ ]:
# Execute experiment
memory_settings = [256, 512, 1024]

for setting in memory_settings:
    FaaSET.reconfigure(function=hello_world_faaset, config={"memory": setting})
    FaaSRunner.experiment(
        function=hello_world_faaset, 
        threads=5,
        runs_per_thread=5,
        payloads=[{"name": "Bob"}],
        experiment_name="memory_test")

results = FaaSRunner.load(function=hello_world_faaset, experiment="memory_test")
results

In [ ]:
# Execute experiment
page_rank_results = FaaSRunner.experiment(
    function=page_rank_faaset,
    threads=30,
    runs_per_thread=2,
    payloads=[{"size": 50000, "loops": 5},
              {"size": 100000, "loops": 5},
              {"size": 150000, "loops": 5}],
    experiment_name="concurrency_test")

page_rank_results

In [ ]:
# Functions and Experiments can be written in the same cell!
@FaaSET.cloud_function(platform="AWS")
def sleeper_faaset(request, context): 
    from SAAF import Inspector
    import time
    inspector = Inspector()
    inspector.inspectAll()
    time.sleep(request['time'])
    inspector.inspectAllDeltas()
    return inspector.finish()

# Test function
print(str(sleeper_faaset({"time": 1}, None)))

# Define and execute experiment
memory_settings = [2048, 4096, 6144]
for setting in memory_settings:
    FaaSET.reconfigure(function=sleeper_faaset, config={"memory": setting})
    FaaSRunner.experiment(
        function=sleeper_faaset, 
        threads=10,
        runs_per_thread=1,
        payloads=[{"time": 5}],
        experiment_name="memory_test")

sleeper_results = FaaSRunner.load(function=sleeper_faaset, experiment="memory_test")
sleeper_results

# General Statistics

In [ ]:
# Run get general statistics
def print_stats(data, metric):
    print("--- Statistics for " + metric + ". ---")
    print("* Min: " + str(data[metric].min()))
    print("* Max: " + str(data[metric].max()))
    print("* Sum: " + str(data[metric].sum()))
    print("* Mean: " + str(data[metric].mean()))
    print("* Standard Deviation: " + str(data[metric].std()))
    print("* Coefficient of Variation: " + str(data[metric].std() / data[metric].mean()))
    
print_stats(page_rank_results, "userRuntime")

In [ ]:
# Run a paired and unpaired T test.

%pip install scipy

import numpy as np
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel

x1 = page_rank_results['cpuUserDelta']
x2 = page_rank_results['cpuIdleDelta']

ttest_ind(x1, x2)
ttest_rel(x1, x2)

# Graph Results

In [ ]:
# Imports for Graphing
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd

In [ ]:
sleeper_results = FaaSRunner.load(function=sleeper_faaset, experiment="memory_test")

# Define figure style
fig = make_subplots(specs = [[{"secondary_y": False}]])
fig.update_layout(
    barmode='stack',
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.47),
    margin=dict(t=0, b=1, l=1, r=1, autoexpand=True),
    font=dict(size=16)
)

workloads = [sleeper_results, sleeper_results, sleeper_results]
names = ["2048 MB", "4096 MB", "6144 MB"]
targetMemory = ["2048", "4096", "6144"]

finalData = pd.DataFrame()
finalData['workloads'] = names

cpuUsers = [] 
cpuIdles = []
cpuKernels = []
runtimes = []

i = 0
for workload in workloads:
    cpuUsers.append(workload[workload['functionMemory'] == targetMemory[i]]['cpuUserDelta'].mean())
    cpuIdles.append(workload[workload['functionMemory'] == targetMemory[i]]['cpuIdleDelta'].mean())
    cpuKernels.append(workload[workload['functionMemory'] == targetMemory[i]]['cpuKernelDelta'].mean())
    runtimes.append(workload[workload['functionMemory'] == targetMemory[i]]['runtime'].mean())
    i += 1

finalData['cpuUser'] = cpuUsers
finalData['cpuIdle'] = cpuIdles
finalData['cpuKernel'] = cpuKernels
finalData['runtime'] = runtimes

fig.add_trace(go.Bar(x = finalData["workloads"],
                y = finalData["cpuKernel"], 
                name = "CPU Kernel", marker_color="rgba(179, 223, 146, 255)"),
                secondary_y=False)

fig.add_trace(go.Bar(x = finalData["workloads"],
                y = finalData["cpuUser"], 
                name = "CPU User", marker_color="rgba(0, 120, 179, 255)"),
                secondary_y=False)

fig.add_trace(go.Bar(x = finalData["workloads"],
                y = finalData["cpuIdle"], 
                name = "CPU Idle", marker_color="rgba(151, 209, 233, 255)"),
                secondary_y=False)

# Set x-axis title
fig.update_xaxes(title_text="Memory Setting")

# Set y-axes titles
fig.update_yaxes(title_text="CPU Time (ms)", secondary_y=False)

fig.show()

In [ ]:
# Import matplotlib and setup display.
import matplotlib.pyplot as plt
%matplotlib inline

# Histogram of runtime
plt.hist(page_rank_results['userRuntime'], 10)